In [ ]:
import numpy as np
import cv2
import socket
from time import sleep

In [ ]:
f = h5py.File('../data/p1ch3/ourpoints.hdf5', 'w')
dset = f.create_dataset('coords', data=points.numpy())
f.close()

In [ ]:
f = h5py.File('/home/shifa/AgentHuman/SeqTrain/data_03663.h5', 'r')
imgs = np.array(f['rgb'])
targets = np.array(f['targets'])

In [ ]:
img = imgs[100]
target = targets[100]
target = target.astype(np.float32)
print(target.shape)
print(target)
# 2 Follow lane, 3 Left, 4 Right, 5 Straight
# -> 0 Follow lane, 1 Left, 2 Right, 3 Straight
command = int(target[24])-2
print('command', command)
# Steer, Gas, Brake (0,1, focus on steer loss)
target_vec = np.zeros((4, 3), dtype=np.float32)
target_vec[command, :] = target[:3]
print(target_vec)
            # in km/h, <90
speed = np.array([target[10]/90, ]).astype(np.float32)
print('speed', speed)
mask_vec = np.zeros((4, 3), dtype=np.float32)
mask_vec[command, :] = 1
print(mask_vec.shape)
plt.imshow(img)
plt.show()
tar = target_vec.reshape(-1)
mask = mask_vec.reshape(-1)
print(speed, tar, mask)

In [ ]:
idx = 0
while(idx<200):
    frame = imgs[idx]
    idx += 1
    cv2.imshow('frame',frame)
    k = cv2.waitKey(100) & 0xff
    if k == 27:
        break
cv2.destroyAllWindows()

In [ ]:
s=socket.socket()
s.connect(('192.168.1.156', 2021))
s.send(b"ONA -180 -180")
sleep(1)
s.send(b"ONA 80 50")
cap = cv2.VideoCapture('http://192.168.1.156:8080/?action=stream')
TURN_LEFT = 2
TURN_RIGHT = 3
s.send(b"ONs")

In [ ]:
speed = 0
turn  = 0
forward = True
file_num = 0
count = 0
target = np.zeros(7, dtype=np.float32)
imgs = np.zeros(200)
targets = np.zeros(200)

while(1):
    count += 1
    ret,frame = cap.read()

    cv2.imshow('frame',frame)
    k = cv2.waitKey(30) & 0xff
    if k == 255:
        continue
    if k == 27:
        break
# 2 Follow lane
# -> 0 Follow lane
# 4*3 -> 1*4 forwards, backward, left, right

# forwards: 0, backward: 1, left: 2, right: 3
    if k == 82:  # forward
        target[0] = 1
        if turn:
            turn = 0
        elif forward:
            speed += 10
        else:
            speed -= 10
            if speed <= 0:
                forward = True
                speed = -speed
    elif k == 84:  # backward
        target[1] = 1
        if turn:
            turn = 0
        elif forward:
            speed -= 10
            if speed <= 0:
                forward = False
                speed = -speed
        else:
            speed += 10
    elif k == 81:  # turn_left
        target[2] = 1
        if turn == TURN_LEFT:
            speed += 10
        elif turn == TURN_RIGHT:
            speed -= 10
            if speed <= 0:
                turn = TURN_LEFT
                speed = -speed
        else:
            turn = TURN_LEFT
    elif k == 83:  # turn_right
        target[3] = 1
        if turn == TURN_RIGHT:
            speed += 10
        elif turn == TURN_LEFT:
            speed -= 10
            if speed <= 0:
                turn = TURN_RIGHT
                speed = -speed
        else:
            turn = TURN_RIGHT

    s.send("ONr {} {}".format(speed if forward else -speed, turn).encode("utf-8"))
    
    if count == 200: # have 200 imgs and targets
        count = 0
        f = h5py.File('data_' + file_num +'.h5', 'w')
        image_set = f.create_dataset('rgb', data=imgs)
        target_set = f.create_dataset('targets', data=targets)
        file_num += 1
    
    img = frame
    
    imgs = np.append(imgs, img)
    
    target[4] = speed
    target[5] = turn
    target[6] = forward
    targets = np.append(targets, target)
    
    f.close()

    if k == ord('l'):
        s.send(b"ONL")
    elif k == ord('r'):
        s.send(b"ONR")
    elif k == ord('u'):
        s.send(b"ONU")
    elif k == ord('d'):
        s.send(b"OND")
    elif k == ord('s'):
        s.send(b"ONs")

s.send(b"ONs")  # Stop running
cv2.destroyAllWindows()

In [ ]:
f = h5py.File('data_0.h5', 'r')
my_imgs = np.array(f['rgb'])
my_targets = np.array(f['targes'])

f.close()
type(my_imgs)
print(my_imgs.shape)
print(my_targets.shape)
display_video(my_imgs, 50)